In [14]:
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from arch.__future__ import reindexing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, Activation, LSTM
from keras.optimizers import Adam
import keras
import datetime
from scipy.stats import norm

In [ ]:
csv_path = "https://raw.githubusercontent.com/curiousily/Deep-Learning-For-Hackers/master/data/3.stock-prediction/BTC-USD.csv"
df = pd.read_csv('tether.csv', parse_dates=['Date'])
df = df.sort_values('Date')
scaler = MinMaxScaler()

close_price = df.Close.values.reshape(-1, 1)

scaled_close = scaler.fit_transform(close_price)
scaled_close = scaled_close[~np.isnan(scaled_close)]
scaled_close = scaled_close.reshape(-1, 1)
SEQ_LEN = 100

def to_sequences(data, seq_len):
    d = []

    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])

    return np.array(d)

def preprocess(data_raw, seq_len, train_split):

    data = to_sequences(data_raw, seq_len)

    num_train = int(train_split * data.shape[0])

    X_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, :]

    X_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, :]

    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test =\
 preprocess(scaled_close, SEQ_LEN, train_split = 0.95)
DROPOUT = 0.2
WINDOW_SIZE = SEQ_LEN - 1
model = keras.Sequential()
model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences=True),
  input_shape=(WINDOW_SIZE, X_train.shape[-1])
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  LSTM((WINDOW_SIZE * 2), return_sequences=True)
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  LSTM(WINDOW_SIZE, return_sequences=False)
))

model.add(Dense(units=1))

model.add(Activation('linear'))
BATCH_SIZE = 64

model.compile(
    loss='mean_absolute_error',
    optimizer='adam'
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=BATCH_SIZE,
    shuffle=False,
    validation_split=0.1
)

y_hat = model.predict(X_test)
y_test_inverse = scaler.inverse_transform(y_test)
y_hat_inverse = scaler.inverse_transform(y_hat)

Epoch 1/50
25/25 [==============================] - 62s 2s/step - loss: 0.1121 - val_loss: 0.0124
Epoch 2/50
25/25 [==============================] - 49s 2s/step - loss: 0.0333 - val_loss: 0.0425
Epoch 3/50
25/25 [==============================] - 47s 2s/step - loss: 0.0271 - val_loss: 0.0017
Epoch 4/50
25/25 [==============================] - 48s 2s/step - loss: 0.0278 - val_loss: 0.0181
Epoch 5/50
25/25 [==============================] - 46s 2s/step - loss: 0.0278 - val_loss: 0.0140
Epoch 6/50
25/25 [==============================] - 48s 2s/step - loss: 0.0216 - val_loss: 0.0020
Epoch 7/50
25/25 [==============================] - 48s 2s/step - loss: 0.0227 - val_loss: 0.0159
Epoch 8/50
25/25 [==============================] - 47s 2s/step - loss: 0.0211 - val_loss: 0.0046
Epoch 9/50
25/25 [==============================] - 49s 2s/step - loss: 0.0225 - val_loss: 0.0041
Epoch 10/50
25/25 [==============================] - 48s 2s/step - loss: 0.0298 - val_loss: 0.0202
Epoch 11/50
25/25 [

In [ ]:
arr = []
for i in range(0, len(y_hat_inverse)-10):
        arr.append((y_hat_inverse[i:i+10].std()))
arr1 = []
for i in range(0, len(y_test_inverse)-10):
        arr1.append((y_test_inverse[i:i+10].std()))
arr=np.array(arr)
arr1=np.array(arr1)
abs((1 - (arr/arr1))).mean()

0.7155478262761814